In [ ]:
# install selenium and other resources for crawling data
!pip install selenium
# install other resources for doing crawling
!apt install chromium-chromedriver

     |████████████████████████████████| 911kB 7.9MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  chromium-browser chromium-browser-l10n chromium-codecs-ffmpeg-extra
Suggested packages:
  webaccounts-chromium-extension unity-chromium-extension
The following NEW packages will be installed:
  chromium-browser chromium-browser-l10n chromium-chromedriver
  chromium-codecs-ffmpeg-extra
0 upgraded, 4 newly installed, 0 to remove and 31 not upgraded.
Need to get 83.2 MB of archives.
After this operation, 282 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-codecs-ffmpeg-extra amd64 89.0.4389.90-0ubuntu0.18.04.2 [1,127 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-browser amd64 89.0.4389.90-0ubuntu0.18.04.2 [73.6 MB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd

In [60]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
from random import randint
import json
import re
import sqlite3

from selenium import webdriver
# Set driver for Chrome
options = webdriver.ChromeOptions()
options.add_argument('-headless') # since we run selenium on Google Colab so we don't want a chrome browser opens, so it will run in the background
options.add_argument('-no-sandbox')

In [61]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [62]:
PATH_TO_DB = '/content/gdrive/MyDrive/Scrape Tiki/'

In [63]:
conn = sqlite3.connect(PATH_TO_DB+'tiki_with_product.db')
cur = conn.cursor()

In [ ]:
# Create table categories in the database using a function
def create_categories_table():
    query = """
        CREATE TABLE IF NOT EXISTS categories (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name VARCHAR(255),
            url TEXT, 
            parent_id INTEGER,
            create_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    """
    try:
        cur.execute(query)
        conn.commit()
    except Exception as err:
        print('ERROR BY CREATE TABLE', err)

In [ ]:
# create_categories_table()

In [ ]:
def select_query(query,conn=conn):
    return pd.read_sql_query(query, conn)

In [69]:
select_query('SELECT * FROM product')

,SKU,id,SKU,Name,url,image_url,price,Discount,Rating,Review,tiki_now,Free_ship,Installment,Free_gift,Under_price,create_at
0,8020626062208,1,8020626062208,Điện Thoại Oppo A12 (3GB/32GB) - Hàng Chính Hãng,https://tiki.vn/dien-thoai-oppo-a12-3gb-32gb-h...,https://salt.tikicdn.com/cache/280x280/ts/prod...,2490000,17.0,4.7,170.0,1,0,0,0,0,2021-04-12 01:27:44
1,8020626062208,2,8020626062208,Điện Thoại Oppo A12 (3GB/32GB) - Hàng Chính Hãng,https://tiki.vn/dien-thoai-oppo-a12-3gb-32gb-h...,https://salt.tikicdn.com/cache/280x280/ts/prod...,2490000,17.0,4.7,170.0,1,0,0,0,0,2021-04-12 01:28:33
2,2480990721662,3,2480990721662,Điện Thoại Realme C11 (2GB/32GB) - Hàng Chính ...,https://tiki.vn/dien-thoai-realme-c11-2gb-32gb...,https://salt.tikicdn.com/cache/280x280/ts/prod...,2149000,20.0,4.8,90.0,1,0,0,0,1,2021-04-12 01:28:33
3,5743069330576,4,5743069330576,Điện Thoại iPhone 12 Pro Max 128GB - Hàng Chín...,https://tiki.vn/dien-thoai-iphone-12-pro-max-1...,https://salt.tikicdn.com/ts/upload/f3/74/46/f4...,29950000,12.0,5.0,427.0,1,1,1,0,0,2021-04-12 01:28:33
4,4132683255463,5,4132683255463,Điện Thoại Vsmart Live 4 (6GB/64GB) - Hàng Chí...,https://tiki.vn/dien-thoai-vsmart-live-4-hang-...,https://salt.tikicdn.com/cache/280x280/ts/prod...,3690000,23.0,4.7,923.0,1,0,0,0,0,2021-04-12 01:28:33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18102,6853185938606,18103,6853185938606,Công tắc điều khiển từ xa RF 4 cổng TX04 sóng ...,https://tiki.vn/cong-tac-dieu-khien-tu-xa-rf-4...,https://salt.tikicdn.com/cache/280x280/ts/prod...,94000,32.0,4.3,7.0,0,0,0,0,1,2021-04-12 02:10:50
18103,4982207067232,18104,4982207067232,Cầu dao đảo chiều 2P 63A 220V SNB1 cầu dao đổi...,https://tiki.vn/cau-dao-dao-chieu-2p-63a-220v-...,https://salt.tikicdn.com/cache/280x280/ts/prod...,165000,3.0,None,None,0,0,0,0,0,2021-04-12 02:10:50
18104,7884939080156,18105,7884939080156,"Công Tắc Điều Khiển Từ Xa Qua Mạng Wifi, 3G So...",https://tiki.vn/cong-tac-dieu-khien-tu-xa-qua-...,https://salt.tikicdn.com/cache/280x280/ts/prod...,139000,39.0,None,None,1,0,0,0,0,2021-04-12 02:10:50
18105,5072580215807,18106,5072580215807,AdCông tắc Wifi thông minh Tuya cảm ứng 4 nút ...,https://tiki.vn//tka.tiki.vn/pixel?data=djAwMX...,https://salt.tikicdn.com/cache/280x280/ts/prod...,357053,40.0,None,None,0,0,0,0,0,2021-04-12 02:10:51


In [ ]:
end = select_query('''
    SELECT id, name, url FROM 
    (
    SELECT *
    FROM categories as a
        LEFT JOIN categories as b ON b.parent_id = a.id
    )
    EXCEPT
    select id, name, url  from 
    (
    SELECT *
    FROM categories as a
        JOIN categories as b ON b.parent_id = a.id
    )
''')

In [ ]:
end.iloc[0].url

'https://tiki.vn/dien-thoai-smartphone/c1795?src=c.1789.hamburger_menu_fly_out_banner'

In [ ]:
def get_url(url):
  driver = webdriver.Chrome('chromedriver',options=options)        # Define the chrome drivers with setting options we define above    
  driver.get(url)                                             # Open the browser again to get web page
  html_data = driver.page_source                                   # After driver.get() is done, you can get back HTML string by using .page_source
  driver.close()                                                   # Close the driver after retrieving the web page

  soup = BeautifulSoup(html_data, 'html.parser') 

  return soup

In [ ]:
def extract_tiki_info(url):
  """ Extract info from all products of a specfic category on Tiki website
      Input: url
      Output: info of products, saved as list of dictionary. If no products shown, return empty list.
  """
  data = []
  index = 1
  soup = get_url(url)

  # FIND ALL PRODUCT ITEMS
  products = soup.find_all('a', {'class':'product-item'})
  all_script = soup.find_all('script', {'type':'application/ld+json'})
  print("BATCH SIZE:", len(products))

  if (soup.find('div', {'class':'style__StyledNotFoundProductView-sc-1uz0b49-0'})):
    print("END PAGE")
  elif len(products):
    # EXTRACT INFO TO DICTIONARY
    for i in products: 
      d = {'name':'','price':'','product_url':'','image':'', 'product_sku':'',
           'tiki_now':'','freeship':'','review':'','rating':'','under_price':'',
           'discount':'','installment':'','gift':''}
      
      try:
        d['name']         = i.find('div',{'class' : 'name'}).text
        d['price']        = int(re.sub('[. ₫]','', i.find('div',{'class':'price-discount__price'}).text))
        d['product_url']  = 'https://tiki.vn' + i['href'] 
        thumbnail         = i.find('div',{'class':'thumbnail'})
        d['image']        = thumbnail.img['src']        
        d['tiki_now']     = bool(i.find('div',{'class':'badge-service'}).find('div',{'class':'item'})) 
        d['freeship']     = bool(i.find('div',{'class':'badge-top'}).text == "Freeship")
        
        if i.find('div',{'class':'review'}):
          d['review']     = int(i.find('div',{'class':'review'}).text.strip('(').strip(')'))
        else:
          d['review'] = "N/A"
        
        d['under_price']  = bool(i.find('div',{'class':'badge-under-price'}).find('div',{'class':'item'}))

        if i.find('div', {'class':'price-discount__discount'}):
          d['discount']   = int(re.sub('[-%]','', i.find('div',{'class':'price-discount__discount'}).text))
        else:
          d['discount']   = "N/A"
        
        d['installment']  = bool(i.find('div',{'class':'badge-benefits'}).img)
        d['gift']         = bool(i.find('div',{'class':'freegift-list'}))

        script = all_script[index]
        dict_content = json.loads(script.text)
        d['product_sku']  = dict_content['sku']
        
        if 'aggregateRating' in dict_content:
          d['rating']     = float(dict_content['aggregateRating']['ratingValue'])
        else:
          d['rating']     = "N/A"

      except Exception as e:
        print(e)

      index += 1
      data.append(d)
        
  return data

In [ ]:
def scrape_tiki(base_url):
  result = []
  page_number = 1
  main, opt = base_url.split('?')
  
  while page_number<=2:
    page = f'?page={page_number}&'
    url = main+page+opt
    print("url =", url)
    data = extract_tiki_info(url)
    if len(data)>0:
      result.extend(data)
    else:
      break
    page_number += 1
    sleep(2)
  return result
  print("****TOTAL = ",len(result))
  

In [ ]:
start = 396
end_num = 700
data_total = []
for index in range(start,end_num):
    print(f'--------Scrapping {index} out of {end_num}-------')
    cat_id = int(end.iloc[index].id)
    url = end.iloc[index].url
    result = scrape_tiki(url)
    # for i in result:
    #     a = Product(cat_id, i['product_sku'],i['name'],i['product_url'],i['image'],i['price'],i['discount'],i['rating'],i['review'],i['tiki_now'],i['freeship'],i['installment'],i['gift'],i['under_price'])
    #     a.save_into_db()
    data_total.extend(result)
data_total
items = pd.DataFrame(data = data_total, columns = data_total[0].keys())
items.to_csv("./result_396_700.csv", index=False)

    

--------Scrapping 396 out of 700-------
url = https://tiki.vn/bot-an-dam/c2606?page=1&src=c.2549.hamburger_menu_fly_out_banner
BATCH SIZE: 48
url = https://tiki.vn/bot-an-dam/c2606?page=2&src=c.2549.hamburger_menu_fly_out_banner
BATCH SIZE: 48
--------Scrapping 397 out of 700-------
url = https://tiki.vn/thuc-pham-che-bien-san/c6571?page=1&src=c.2549.hamburger_menu_fly_out_banner
BATCH SIZE: 50
url = https://tiki.vn/thuc-pham-che-bien-san/c6571?page=2&src=c.2549.hamburger_menu_fly_out_banner
BATCH SIZE: 48
--------Scrapping 398 out of 700-------
url = https://tiki.vn/sua-cong-thuc-pha-san/c10419?page=1&src=c.2549.hamburger_menu_fly_out_banner
BATCH SIZE: 49
url = https://tiki.vn/sua-cong-thuc-pha-san/c10419?page=2&src=c.2549.hamburger_menu_fly_out_banner
BATCH SIZE: 48
--------Scrapping 399 out of 700-------
url = https://tiki.vn/banh-an-dam/c6570?page=1&src=c.2549.hamburger_menu_fly_out_banner
BATCH SIZE: 48
url = https://tiki.vn/banh-an-dam/c6570?page=2&src=c.2549.hamburger_menu_fly_

In [ ]:
# Instead of using a function to do CRUD on database,
# creating a class Category is preferred
# attributes: name, url, parent_id
# instance method: save_into_db()
class Category:
    def __init__(self, name, url, parent_id=None, cat_id=None):
        self.cat_id = cat_id
        self.name = name
        self.url = url
        self.parent_id = parent_id

    def __repr__(self):
        return f"ID: {self.cat_id}, Name: {self.name}, URL: {self.url}, Parent: {self.parent_id}"

    def save_into_db(self):
        query = """
            INSERT INTO categories (name, url, parent_id)
            VALUES (?, ?, ?);
        """
        val = (self.name, self.url, self.parent_id)
        try:
            cur.execute(query, val)
            self.cat_id = cur.lastrowid
            conn.commit()
        except Exception as err:
            print('ERROR BY INSERT:', err)


In [ ]:
# Create table categories in the database using a function
def create_product_table():
    query = """
        CREATE TABLE IF NOT EXISTS Product(
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            SKU TEXT,
            Name TEXT,
            url TEXT,
            image_url TEXT,
            price INTEGER,
            Discount TEXT,
            Rating TEXT,
            Review TEXT,
            tiki_now TEXT,
            Free_ship TEXT,
            Installment TEXT,
            Free_gift TEXT,
            Under_price TEXT,
            create_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    """
    try:
        cur.execute(query)
        conn.commit()
    except Exception as err:
        print('ERROR BY CREATE TABLE', err)

create_product_table()

In [ ]:
# Instead of using a function to do CRUD on database,
# creating a class Category is preferred
# attributes: name, url, parent_id
# instance method: save_into_db()
class Product:
    def __init__(self, SKU, name, url, image_url, price, discount, rating, review, tiki_now,free_ship,installment, free_gift, under_price):
        self.SKU = SKU
        self.name = name
        self.url = url
        self.image_url = image_url
        self.price = price
        self.discount = discount
        self.rating = rating
        self.review = review
        self.tiki_now = tiki_now
        self.free_ship = free_ship
        self.installment = installment
        self.free_gift = free_gift
        self.under_price = under_price

    def __repr__(self):
        return f"Name: {self.name}, URL: {self.url}"

    def save_into_db(self):
        query = """
            INSERT INTO Product ( SKU, name, url, image_url, price, discount, rating, review, tiki_now,free_ship,installment, free_gift, under_price)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
        """
        val = (self.SKU, self.name, self.url, self.image_url, self.price, self.discount, self.rating, self.review, self.tiki_now, self.free_ship, self.installment, self.free_gift, self.under_price)
        try:
            cur.execute(query, val)
            conn.commit()
        except Exception as err:
            print('ERROR BY INSERT:', err)


In [ ]:
import os
import glob
os.chdir("/content/gdrive/MyDrive/Scrape Tiki/")
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
combined_csv.to_csv( "combined_csv.csv", index=False, encoding='utf-8-sig')


In [ ]:
# empty our category set
PRODUCT_SET = set()

In [ ]:
def can_add_to_pro_set(product_sku,save=False):
  if product_sku not in PRODUCT_SET:
    if save:
      PRODUCT_SET.add(product_sku)
    #   print(f'Added "{product_sku}" to PRODUCT_SET')
    return True
  return False


In [ ]:
def get_product(product,save_db=False):
    result=[]
    for i in range(len(product)):
        _=can_add_to_pro_set(product[i]['product_sku'],save_db)

        product_detail = Product(product[i]['product_sku'],product[i]['name'], product[i]['product_url'], product[i]['image'], product[i]['price'], product[i]['discount'], product[i]['rating'], product[i]['review'], product[i]['tiki_now'],product[i]['freeship'],product[i]['installment'], product[i]['gift'], product[i]['under_price'])
        print(f'---{i} in {len(product)}---')
        if save_db:
            product_detail.save_into_db()
        result.append(product_detail)
    return result

In [ ]:
product = pd.read_csv('product.csv')

In [ ]:
product = product.to_dict('index')

In [ ]:
get_product(product,save_db=True)

---0 in 167465---
---1 in 167465---
---2 in 167465---
---3 in 167465---
---4 in 167465---
---5 in 167465---
---6 in 167465---
---7 in 167465---
---8 in 167465---
---9 in 167465---
---10 in 167465---
---11 in 167465---
---12 in 167465---
---13 in 167465---
---14 in 167465---
---15 in 167465---
---16 in 167465---
---17 in 167465---
---18 in 167465---
---19 in 167465---
---20 in 167465---
---21 in 167465---
---22 in 167465---
---23 in 167465---
---24 in 167465---
---25 in 167465---
---26 in 167465---
---27 in 167465---
---28 in 167465---
---29 in 167465---
---30 in 167465---
---31 in 167465---
---32 in 167465---
---33 in 167465---
---34 in 167465---
---35 in 167465---
---36 in 167465---
---37 in 167465---
---38 in 167465---
---39 in 167465---
---40 in 167465---
---41 in 167465---
---42 in 167465---
---43 in 167465---
---44 in 167465---
---45 in 167465---
---46 in 167465---
---47 in 167465---
---48 in 167465---
---49 in 167465---
---50 in 167465---
---51 in 167465---
---52 in 167465---
---

KeyboardInterrupt: ignored

In [ ]:
# import re

# # get_sub_categories() given a parent category
# def get_sub_categories(parent_category, save_db=False):
#     parent_url = parent_category.url
#     result = []

#     try:
#         soup = get_url(parent_url)
#         for a in soup.find_all('a', {'class':'item item--category '}):
#             name = a.text.strip()
#             if can_add_to_cat_set(name,save_db): 
#               sub_url = a['href']
#               cat = Category(name, sub_url, parent_category.cat_id) # we now have parent_id, which is cat_id of parent category
#               if save_db:
#                   cat.save_into_db()
#               result.append(cat)
#     except Exception as err:
#         print('ERROR IN GETTING SUB CATEGORIES:', err)
#     return result

In [ ]:
# def get_all_categories(categories,save_db):
#     categories = categories.copy()
#     while len(categories):
#         cat_to_crawl = categories[0]
#         print(f'Getting {cat_to_crawl} sub-categories...')
#         sub_categories = get_sub_categories(cat_to_crawl, save_db=save_db)
#         print(f'Finished! {cat_to_crawl.name} has {len(sub_categories)} sub-categories')
#         categories+=sub_categories
#         del categories[0]

In [ ]:
# # drop the whole table to clean things up
# cur.execute('DROP TABLE categories;')
# conn.commit()

# # re-create our category table again
# create_categories_table()

In [ ]:
# # empty our category set
# CATEGORY_SET= set()

In [ ]:
# main_categories = [{'Name': 'Điện Thoại - Máy Tính Bảng', 'URL': 'https://tiki.vn/dien-thoai-may-tinh-bang/c1789?src=c.1789.hamburger_menu_fly_out_banner'},
# {'Name': 'Điện Tử - Điện Lạnh', 'URL': 'https://tiki.vn/tivi-thiet-bi-nghe-nhin/c4221?src=c.4221.hamburger_menu_fly_out_banner'}, 
# {'Name': 'Phụ Kiện - Thiết Bị Số', 'URL': 'https://tiki.vn/thiet-bi-kts-phu-kien-so/c1815?src=c.1815.hamburger_menu_fly_out_banner'},
# {'Name': 'Laptop - Thiết bị IT', 'URL': 'https://tiki.vn/laptop-may-vi-tinh/c1846?src=c.1846.hamburger_menu_fly_out_banner'},
# {'Name': 'Máy Ảnh - Quay Phim', 'URL': 'https://tiki.vn/may-anh/c1801?src=c.1801.hamburger_menu_fly_out_banner'}, 
# {'Name': 'Điện Gia Dụng', 'URL': 'https://tiki.vn/dien-gia-dung/c1882?src=c.1882.hamburger_menu_fly_out_banner'}, 
# {'Name': 'Nhà Cửa Đời Sống', 'URL': 'https://tiki.vn/nha-cua-doi-song/c1883?src=c.1883.hamburger_menu_fly_out_banner'}, 
# {'Name': 'Hàng Tiêu Dùng - Thực Phẩm', 'URL': 'https://tiki.vn/bach-hoa-online/c4384?src=c.4384.hamburger_menu_fly_out_banner'}, 
# {'Name': 'Đồ chơi, Mẹ & Bé', 'URL': 'https://tiki.vn/me-va-be/c2549?src=c.2549.hamburger_menu_fly_out_banner'}, 
# {'Name': 'Làm Đẹp - Sức Khỏe', 'URL': 'https://tiki.vn/lam-dep-suc-khoe/c1520?src=c.1520.hamburger_menu_fly_out_banner'},
# {'Name': 'Thể Thao - Dã Ngoại', 'URL': 'https://tiki.vn/the-thao/c1975?src=c.1975.hamburger_menu_fly_out_banner'},
# {'Name': 'Xe Máy, Ô tô, Xe Đạp', 'URL': 'https://tiki.vn/o-to-xe-may-xe-dap/c8594?src=c.8594.hamburger_menu_fly_out_banner'},
# {'Name': 'Hàng quốc tế', 'URL': 'https://tiki.vn/hang-quoc-te/c17166?src=c.17166.hamburger_menu_fly_out_banner'}, 
# {'Name': 'Sách, VPP & Quà Tặng', 'URL': 'https://tiki.vn/nha-sach-tiki/c8322?src=c.8322.hamburger_menu_fly_out_banner'}, 
# {'Name': 'Voucher - Dịch Vụ - Thẻ Cào', 'URL': 'https://tiki.vn/voucher-dich-vu/c11312?src=c.11312.hamburger_menu_fly_out_banner'}]

In [ ]:
# main_cat_objs = get_main_category(main_categories,save_db=True)

In [ ]:
# get_all_categories(main_cat_objs,save_db=True)

In [ ]:
# select_query('''
# SELECT categories.id AS I_0, categories.name AS N_0, a.id AS I_1, a.name AS N_1, b.id AS I_2, b.name AS N_2, c.id AS I_3, c.name AS N_3, d.id AS I_4, d.name AS N_4, e.id AS I_5, e.name AS N_5
# FROM categories
#     LEFT JOIN categories as a ON a.parent_id = categories.id
#     LEFT JOIN categories as b ON b.parent_id = a.id
#     LEFT JOIN categories as c ON c.parent_id = b.id
#     LEFT JOIN categories as d ON d.parent_id = c.id
#     LEFT JOIN categories as e ON e.parent_id = d.id

# ORDER BY I_5 DESC,I_4 DESC,I_3 DESC,I_2 DESC,I_1 DESC,I_0 DESC 
# ''')

In [ ]:
cur.execute('DROP TABLE Product;')
conn.commit()
# cur.execute('DROP TABLE categories;')
# conn.commit()

In [ ]:
# cur.execute('DROP TABLE product;')
# conn.commit()

In [ ]:
# import csv
# data = select_query('SELECT category_id, SKU, name, url, image_url, price, discount, rating, review, tiki_now, free_ship, installment, free_gift, under_price, create_at  FROM Product')
# data.to_csv('./data_0_100.csv',index = False)

In [71]:
select_query('SELECT * FROM product ORDER BY rating DESC')

,id,SKU,Name,url,image_url,price,Discount,Rating,Review,tiki_now,Free_ship,Installment,Free_gift,Under_price,create_at
0,4,5743069330576,Điện Thoại iPhone 12 Pro Max 128GB - Hàng Chín...,https://tiki.vn/dien-thoai-iphone-12-pro-max-1...,https://salt.tikicdn.com/ts/upload/f3/74/46/f4...,29950000,12.0,5.0,427.0,1,1,1,0,0,2021-04-12 01:28:33
1,27,6358664703313,Điện Thoại Xiaomi Redmi 9 - Hàng Chính Hãng,https://tiki.vn/dien-thoai-xiaomi-redmi-9-hang...,https://salt.tikicdn.com/cache/280x280/ts/prod...,2890000,19.0,5.0,7.0,0,0,0,0,0,2021-04-12 01:28:33
2,31,1423368008225,Điện thoại Vsmart Star 4 (4GB/64GB) - Hàng Chí...,https://tiki.vn/dien-thoai-vsmart-star-4-4gb-6...,https://salt.tikicdn.com/cache/280x280/ts/prod...,2210000,18.0,5.0,4.0,0,0,0,0,1,2021-04-12 01:28:33
3,38,2016189653911,Điện Thoại Samsung Galaxy A11 (32GB/3GB) - Hàn...,https://tiki.vn/dien-thoai-samsung-galaxy-a11-...,https://salt.tikicdn.com/ts/upload/f3/74/46/f4...,2645000,28.0,5.0,7.0,1,1,0,0,0,2021-04-12 01:28:33
4,40,5566079230069,Điện Thoại Oppo Reno 5 (8GB/128G) - Hàng Chính...,https://tiki.vn/dien-thoai-oppo-reno-5-8gb-128...,https://salt.tikicdn.com/cache/280x280/ts/prod...,7250000,19.0,5.0,22.0,1,0,1,0,0,2021-04-12 01:28:33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18102,18102,7170757394548,AdCông tắc Wifi thông minh cảm ứng 4 nút điều ...,https://tiki.vn//tka.tiki.vn/pixel?data=djAwMf...,https://salt.tikicdn.com/cache/280x280/ts/prod...,350000,42.0,None,None,0,0,0,0,0,2021-04-12 02:10:50
18103,18104,4982207067232,Cầu dao đảo chiều 2P 63A 220V SNB1 cầu dao đổi...,https://tiki.vn/cau-dao-dao-chieu-2p-63a-220v-...,https://salt.tikicdn.com/cache/280x280/ts/prod...,165000,3.0,None,None,0,0,0,0,0,2021-04-12 02:10:50
18104,18105,7884939080156,"Công Tắc Điều Khiển Từ Xa Qua Mạng Wifi, 3G So...",https://tiki.vn/cong-tac-dieu-khien-tu-xa-qua-...,https://salt.tikicdn.com/cache/280x280/ts/prod...,139000,39.0,None,None,1,0,0,0,0,2021-04-12 02:10:50
18105,18106,5072580215807,AdCông tắc Wifi thông minh Tuya cảm ứng 4 nút ...,https://tiki.vn//tka.tiki.vn/pixel?data=djAwMX...,https://salt.tikicdn.com/cache/280x280/ts/prod...,357053,40.0,None,None,0,0,0,0,0,2021-04-12 02:10:51
